In [1]:
import numpy as np
import pandas as pd
import json
import altair as alt
#from vega_datasets import data
alt.renderers.set_embed_options(padding={"left": 0, "right": 0, "bottom": 0, "top": 0})

#alt.renderers.enable('notebook')
alt.data_transformers.enable('json') 

#alt.renderers.enable('json')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [63]:
df=pd.read_json(r'C:\Users\Felipe\Documents\GitHub\Trabalho_Vis\Camara_Deputados\altair-data-a77df692dddb934c30bc41bdea263320.json')#,error_bad_lines=False)

In [4]:
df=pd.read_json(r'C:\Users\Felipe\Documents\GitHub\Trabalho_Vis\Camara_Deputados\data\topicos_tudo.json')#,error_bad_lines=False)
#df = df[df['Dominant_Topic']!='']
#df = df[df['Data']>='2012-01-01'].reset_index(drop=True)



df['Bar'] = 1

df.loc[df['Partido'] == 'SOLIDARIEDADE', 'Partido'] = 'SD'
df.loc[df['Partido'] == 'SDD', 'Partido'] = 'SD'
df.loc[df['Partido'] == 'PMDB', 'Partido'] = 'MDB'
df.loc[df['Partido'] == 'PTN', 'Partido'] = 'PODE'
df.loc[df['Partido'] == 'PEN', 'Partido'] = 'PATRIOTA'
df.loc[df['Partido'] == 'PL', 'Partido'] = 'PR'
df.loc[df['Partido'] == 'CIDADANIA', 'Partido'] = 'PPS'

df.loc[df['Dominant_Topic'] == 'Rio de Janeiro/Petrobras/Petróleo', 'Dominant_Topic'] = 'Petrobras/Petróleo/RJ'
df.loc[df['Dominant_Topic'] == 'Agricultura', 'Dominant_Topic'] = 'Agricultura/RS'
df = df[df['Dominant_Topic']!='0.0']
df = df[df['Dominant_Topic']!='1.0']
df = df.drop(df[df['Dominant_Topic'] == '4.0'][df[df['Dominant_Topic'] == '4.0']['Topic_Perc_Contrib']<0.25].index)
df = df.drop(df[df['Dominant_Topic'] == '6.0'][df[df['Dominant_Topic'] == '6.0']['Topic_Perc_Contrib']<0.15].index)
df.loc[df['Dominant_Topic'] == '6.0', 'Dominant_Topic'] = 'Desenvolvimento Econômico'
df.loc[df['Dominant_Topic'] == '7.0', 'Dominant_Topic'] = 'Município'
df = df.drop(df[df['Dominant_Topic'] == '11.0'][df[df['Dominant_Topic'] == '11.0']['Topic_Perc_Contrib']<0.25].index)
df = df[df['Dominant_Topic']!='13.0']
df = df[df['Dominant_Topic']!='15.0']
df = df[df['Dominant_Topic']!='18.0']
df.loc[df['Dominant_Topic'] == '20.0', 'Dominant_Topic'] = 'Idoso/Juventude/Minorias'
df = df[df['Dominant_Topic']!='24.0']
df.loc[df['Dominant_Topic'] == '28.0', 'Dominant_Topic'] = 'Cultura'
df.loc[df['Dominant_Topic'] == '30.0', 'Dominant_Topic'] = 'Votação'
df.loc[df['Dominant_Topic'] == '31.0', 'Dominant_Topic'] = 'Homenagem'
df = df[df['Dominant_Topic']!='37.0']


df = df[df['Dominant_Topic']!='Votação da Câmara dos Deputados']
df = df[df['Dominant_Topic']!='Votação']
df.loc[df['Dominant_Topic'] == '11.0', 'Dominant_Topic'] = 'Ativ. Parlamentar'
df.loc[df['Dominant_Topic'] == '4.0', 'Dominant_Topic'] = 'Homenagem'

df['Textbr'] = df.apply(line_break,axis=1)

df = df.drop(columns=['Topic_Perc_Contrib','Keywords','Hora','Text'])

#df = df[:1000]
url = 'topicos.json'
df.to_json(url,orient = 'records')

In [6]:
eventos = pd.DataFrame([
    {
        "start": "2014-03-01",
        "end": "2019-12-19",
        "evento": "Lava Jato"
    },
    {
        "start": "2018-04-07",
        "end": "2019-11-08",
        "evento": "Lula é preso"
    },
    {
        "start": "2013-06-02",
        "end": "2013-07-01",
        "evento": "Protestos 2013"
    },
    {
        "start": "2016-04-15",
        "end": "2016-08-25",
        "evento": "Impeachment"
    },
    {
        "start": "2017-05-17",
        "end": "2017-07-17",
        "evento": "Friboigate"
    }
])

In [66]:
stack_input = ['center','normalized','zero']

In [57]:
pts=alt.selection(type="multi",encodings=['x'])
selection = alt.selection_multi(fields=['Dominant_Topic'], bind='legend')
brush = alt.selection(type='interval', encodings=['x'])

stack_dropdown = alt.binding_select(options=stack_input)
################################3

base_bar = alt.Chart('data/topicos.json').transform_filter(brush).transform_aggregate(num_par='count()',groupby=['Dominant_Topic', 'Bar']).\
encode(alt.X('Dominant_Topic:O', scale=alt.Scale(paddingInner=0), axis=alt.Axis(orient='top',title='Tema dos Discursos')),alt.Y('Bar:O', scale=alt.Scale(paddingInner=0),axis=None))

heatmap_bar = base_bar.mark_rect().encode(color=alt.condition(pts, alt.ColorValue("red"),alt.ColorValue("grey"))).\
properties(width=1000,height=25).add_selection(pts)

text_bar = base_bar.mark_text(size=10,baseline='middle').encode(text='num_par:Q',color=alt.condition(alt.datum.num_par > 10000,alt.value('black'),alt.value('white')))

bar = heatmap_bar + text_bar


#####################


#tooltip=[alt.Tooltip('Dominant_Topic:N',title='Tópico')]
base = alt.Chart('data/topicos.json').mark_area(interpolate='monotone').encode(
    alt.X('yearmonth(Data):T',axis=alt.Axis(format=("%b %Y"),title=None,domain=False,tickSize=0)),
    alt.Y('sum(Word_Count):Q',stack = 'zero',axis=alt.Axis(title=None)),alt.Color('Dominant_Topic:N',scale=alt.Scale(scheme='dark2'),legend=alt.Legend(orient='right'))
,opacity=alt.condition(selection, alt.value(1), alt.value(0.2))).properties(width=1000,height=400).transform_filter(pts).add_selection(selection)

upper = base.encode(alt.X('yearmonth(Data):T', scale=alt.Scale(domain=brush), axis=alt.Axis(title=None)),\
                    alt.Y('sum(Word_Count):Q',stack = variaveis['Stack'][0],axis= alt.Axis(title ='Quantidade de Palavras')),tooltip=['Dominant_Topic:N'])

lower = base.encode(alt.Color('Dominant_Topic:N',scale=alt.Scale(scheme='dark2'),legend=None)).properties(height=60).add_selection(brush)

rule=alt.Chart(eventos).mark_rule(color='black',strokeWidth=1).encode(x='start:T')
text=alt.Chart(eventos).mark_text(align='left',baseline='top',dy=-195,size=9).encode(alt.X('start:T',scale=alt.Scale(domain=brush)),text='evento',color=alt.value('#000000'))


################################


pts2=alt.selection(type="single",encodings=['x'])

pts3=alt.selection(type="single",encodings=['x'])

base_bar2 = alt.Chart('data/topicos.json').transform_filter(pts).transform_filter(brush).transform_aggregate(num_par='count()',groupby=['Partido', 'Bar']).\
encode(alt.X('Partido:O', scale=alt.Scale(paddingInner=0)),alt.Y('Bar:O', scale=alt.Scale(paddingInner=0),axis=None))

heatmap_bar2 = base_bar2.mark_rect().encode(color=alt.condition(pts2, alt.ColorValue("#393b79"), alt.ColorValue("grey"))).\
properties(width=500,height=25).add_selection(pts2)

text_bar2 = base_bar2.mark_text(size=8,baseline='middle').encode(text='num_par:Q',color=alt.condition(alt.datum.num_par > 10000,alt.value('black'),alt.value('white')))

bar2 = heatmap_bar2 + text_bar2

base_bar3 = alt.Chart('data/topicos.json').transform_filter(pts).transform_filter(brush).transform_aggregate(num_par='count()',groupby=['Partido', 'Bar']).\
encode(alt.X('Partido:O', scale=alt.Scale(paddingInner=0)),alt.Y('Bar:O', scale=alt.Scale(paddingInner=0),axis=None))

heatmap_bar3 = base_bar3.mark_rect().encode(color=alt.condition(pts3, alt.ColorValue("#393b79"), alt.ColorValue("grey"))).\
properties(width=500,height=25).add_selection(pts3)

text_bar3 = base_bar3.mark_text(size=8,baseline='middle').encode(text='num_par:Q',color=alt.condition(alt.datum.num_par > 10000,alt.value('black'),alt.value('white')))

bar3 = heatmap_bar3 + text_bar3

############

pts4=alt.selection(type="single",encodings=['y'])

bar4 = alt.Chart('data/topicos.json').mark_rect().encode(alt.X('Dominant_Topic:N',scale=alt.Scale(domain=pts)),\
                                                        alt.Y('Nome:N',axis=alt.Axis(orient='right')),color=alt.Color('Word_Count:Q',scale=alt.Scale(scheme='plasma'),legend=alt.Legend(orient='bottom')),tooltip=[alt.Tooltip('Nome:N',title='Nome'),\
                                                                                                                                                  alt.Tooltip('Partido:N',title='Partido')]).\
properties().transform_filter(pts2).add_selection(pts4).transform_filter(pts)

upper2= base.encode(alt.X('yearmonth(Data):T', scale=alt.Scale(domain=brush), axis=alt.Axis(title=None)),alt.Y('sum(Word_Count):Q',axis=None,stack = variaveis['Stack'][0]),alt.Color('Dominant_Topic:N',legend=None,scale=alt.Scale(scheme='dark2')),tooltip=[alt.Tooltip('Dominant_Topic:N',title='Tópico')]).transform_filter(pts2).transform_filter(pts).properties(width=500,height=200)#.transform_filter(pts4)


pts5=alt.selection(type="single",encodings=['y'])
bar5 = alt.Chart('data/topicos.json').mark_rect().encode(alt.X('Dominant_Topic:N'),\
                                                        alt.Y('Nome:N'),color=alt.Color('Word_Count:Q',scale=alt.Scale(scheme='plasma'),legend=alt.Legend(orient='bottom')),tooltip=[alt.Tooltip('Nome:N',title='Nome'),\
                                                                                                                                                  alt.Tooltip('Partido:N',title='Partido')]).\
properties().transform_filter(pts3).add_selection(pts5).transform_filter(pts)#.properties(width=500,height=5000)


upper3= base.encode(alt.X('yearmonth(Data):T', scale=alt.Scale(domain=brush), axis=alt.Axis(title=None)),alt.Y('sum(Word_Count):Q',axis=None,stack = variaveis['Stack'][0]),alt.Color('Dominant_Topic:N',legend=None,scale=alt.Scale(scheme='dark2')),tooltip=[alt.Tooltip('Dominant_Topic:N',title='Tópico')]).transform_filter(pts3).transform_filter(pts).properties(width=500,height=200)#.transform_filter(pts5)#.configure_legend(orient='bottom')

##############################################

display_text_left = alt.Chart('data/topicos.json').mark_text(fontSize=10,lineBreak='<lbr>',align='left',baseline='top',dy=-95,dx=-245).encode(alt.X('yearmonth(Data):T',axis=None),\
                                                  alt.Y('Textbr:N',axis=None),\
                                                  text = 'Textbr:N').properties(width=500,height=200).transform_filter(pts2).transform_filter(pts).transform_filter(brush)


display_text_right = alt.Chart('data/topicos.json').mark_text(fontSize=10,lineBreak='<lbr>',align='left',baseline='top',dy=-95,dx=-245).encode(alt.X('yearmonth(Data):T',axis=None),\
                                                  alt.Y('Textbr:N',axis=None),\
                                                  text = 'Textbr:N').properties(width=500,height=200).transform_filter(pts3).transform_filter(pts).transform_filter(brush)

total_chart = (bar & (rule+text+upper) & (rule+lower) & (bar2|bar3) & alt.hconcat(upper2 , upper3).resolve_scale(y='shared') & (display_text_left.transform_sample(1)|display_text_right.transform_sample(1))).configure_view(strokeWidth=0)# & alt.hconcat(bar4, bar5,spacing=200)#.transform_sample(20) #& text_display.transform_sample(20)

In [58]:
total_chart

alt.VConcatChart(...)

In [62]:
total_chart.save('topicos.html')

In [3]:
def add_br(s, old, new, occurrence):
    
    li = s.rsplit(old, occurrence)
    return new.join(li)

def line_break(row,line_len=80):
    
    i = 0
    d = True
    new_text = row['Dominant_Topic'] + '<lbr>' +'<lbr>' +\
    'Data: '+ row['Data'] + '<lbr>' + '  Parlamentar: ' + row['Nome'] + '  Partido: ' + row['Partido'] + '  Estado: ' +\
    row['Estado'] +'<lbr>' + '<lbr>' + '<lbr>'
    
    while d:

        linha = row['Text'][i*line_len:(i+1)*line_len]
        linha = add_br(linha,' ','<lbr>',1)
        if linha.count(' ') == 0:
            d = False
            new_text += linha
            break
        
        new_text += linha
      
        i +=1
    
    return new_text